In [1]:

!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.7MB 61kB/s 
     |████████████████████████████████| 204kB 27.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=e16ab1c65579fa89f855edb0c9065c4dd87b1fa9b7909ff928a580da15bea7e1
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 33 not upgraded.
Need to get 35.8 MB of archives.
After this 

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# Organización de Datos 75.06. Segundo Cuatrimestre de 2017. Examen parcial, segunda oportunidad:


### 1) A partir de la plataforma online (e-shop) de los países en los que opera, Nintendo tiene información de ventas de videojuegos diarias digitales por país en el siguiente RDD: (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias). Por otro lado se tienen otro RDD que tiene información de todos los videojuegos que se venden en su plataforma con el siguiente formato (id_videojuego, titulo, rating_pegi, rating_esbr). Tener en cuenta que un mismo videojuego se puede vender en distintos países y esos nos permitirá obtener métricas a nivel global. Con esta información escribir un programa en pySpark que permita:
### a) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales
### b) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias) (**) (15 pts) 


In [6]:
## (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias).
ventas = [(1234, 5, '2018-01-01', 5000, 2300),
          (1234, 1, '2018-02-01', 4500, 2300),
          (1234, 3, '2018-03-01', 2000, 2100),
          (1422, 1, '2018-02-01', 7000, 300),
          (1422, 2, '2018-03-01', 3400, 2100),
          (1233, 2, '2018-02-01', 1200, 400),
          (1231, 5, '2018-01-01', 1600, 1300),
          (2223, 1, '2018-01-01', 2200, 2100),
          (2223, 2, '2018-04-01', 4200, 4200),
          (2223, 3, '2018-03-01', 9800, 1600),
          (1212, 2, '2018-02-01', 1100, 1100),
          (1414, 3, '2018-03-01', 5100, 2100),
          (4142, 5, '2018-01-01', 4300, 2300),
          (4142, 2, '2018-01-01', 3300, 200),
          (4142, 4, '2018-02-01', 6500, 2400),
          (3341, 1, '2018-02-01', 5110, 4900)]

## (id_videojuego, titulo, rating_pegi, rating_esbr)
videojuegos = [(1234,'Monkey Island', 6, 6.5),
               (1422,'Tomb Rider', 4, 4),
               (1233,'FarCry 2', 7, 7.5),
               (1231,'Crash', 5, 5.2),
               (2223,'Abe Oddisey', 9, 8.5),
               (1212,'The Phantom Menace', 6.5, 6),
               (1414,'Sugar Candy', 6, 6),
               (4142,'Fatal Frame 11', 7, 7),
               (3341,'Final Fantasy', 9.1, 8.3)]

In [7]:
ventasRDD = sc.parallelize(ventas)
videojuegosRDD = sc.parallelize(videojuegos)

In [9]:
ventasRDD.collect()

[(1234, 5, '2018-01-01', 5000, 2300),
 (1234, 1, '2018-02-01', 4500, 2300),
 (1234, 3, '2018-03-01', 2000, 2100),
 (1422, 1, '2018-02-01', 7000, 300),
 (1422, 2, '2018-03-01', 3400, 2100),
 (1233, 2, '2018-02-01', 1200, 400),
 (1231, 5, '2018-01-01', 1600, 1300),
 (2223, 1, '2018-01-01', 2200, 2100),
 (2223, 2, '2018-04-01', 4200, 4200),
 (2223, 3, '2018-03-01', 9800, 1600),
 (1212, 2, '2018-02-01', 1100, 1100),
 (1414, 3, '2018-03-01', 5100, 2100),
 (4142, 5, '2018-01-01', 4300, 2300),
 (4142, 2, '2018-01-01', 3300, 200),
 (4142, 4, '2018-02-01', 6500, 2400),
 (3341, 1, '2018-02-01', 5110, 4900)]

In [10]:
videojuegosRDD.collect()

[(1234, 'Monkey Island', 6, 6.5),
 (1422, 'Tomb Rider', 4, 4),
 (1233, 'FarCry 2', 7, 7.5),
 (1231, 'Crash', 5, 5.2),
 (2223, 'Abe Oddisey', 9, 8.5),
 (1212, 'The Phantom Menace', 6.5, 6),
 (1414, 'Sugar Candy', 6, 6),
 (4142, 'Fatal Frame 11', 7, 7),
 (3341, 'Final Fantasy', 9.1, 8.3)]

In [11]:
#a) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato:
#(id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales
ventasGlobalesAux = ventasRDD.map(lambda x: (x[0],(x[3])))
ventasGlobalesAux.collect()

[(1234, 5000),
 (1234, 4500),
 (1234, 2000),
 (1422, 7000),
 (1422, 3400),
 (1233, 1200),
 (1231, 1600),
 (2223, 2200),
 (2223, 4200),
 (2223, 9800),
 (1212, 1100),
 (1414, 5100),
 (4142, 4300),
 (4142, 3300),
 (4142, 6500),
 (3341, 5110)]

In [13]:
ventasGlobalesTotales = ventasGlobalesAux.reduceByKey(lambda x,y: x+y)
ventasGlobalesTotales.collect()

[(1234, 11500),
 (1422, 10400),
 (1212, 1100),
 (1414, 5100),
 (4142, 14100),
 (1233, 1200),
 (1231, 1600),
 (2223, 16200),
 (3341, 5110)]

In [14]:
videojuegosLista = videojuegosRDD.map(lambda x:(x[0],x[1]))
videojuegosLista.collect()

[(1234, 'Monkey Island'),
 (1422, 'Tomb Rider'),
 (1233, 'FarCry 2'),
 (1231, 'Crash'),
 (2223, 'Abe Oddisey'),
 (1212, 'The Phantom Menace'),
 (1414, 'Sugar Candy'),
 (4142, 'Fatal Frame 11'),
 (3341, 'Final Fantasy')]

In [15]:
ventasGlobalesVideoJuegos = videojuegosLista.join(ventasGlobalesTotales)
ventasGlobalesVideoJuegos.collect()

[(1212, ('The Phantom Menace', 1100)),
 (1233, ('FarCry 2', 1200)),
 (3341, ('Final Fantasy', 5110)),
 (1234, ('Monkey Island', 11500)),
 (1422, ('Tomb Rider', 10400)),
 (1414, ('Sugar Candy', 5100)),
 (4142, ('Fatal Frame 11', 14100)),
 (1231, ('Crash', 1600)),
 (2223, ('Abe Oddisey', 16200))]

In [21]:
ventasGlobalesRDD = (ventasGlobalesVideoJuegos.map(lambda x: (x[0],x[1][0],x[1][1]))).sortBy(lambda x:-x[2])
ventasGlobalesRDD.take(1)

[(2223, 'Abe Oddisey', 16200)]

In [22]:
#b) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión
#(es decir, mayor total_ventas_diarias / visitas_diarias) (**) (15 pts)
masVendidoPaises = ventasRDD.filter(lambda x: x[0]==2223)
masVendidoPaises.collect()

[(2223, 1, '2018-01-01', 2200, 2100),
 (2223, 2, '2018-04-01', 4200, 4200),
 (2223, 3, '2018-03-01', 9800, 1600)]

In [26]:
tasaConversionMasVendidoPorPais = (masVendidoPaises.map(lambda x: (x[1], x[4]/x[3]))).sortBy(lambda x: -x[1])
tasaConversionMasVendidoPorPais.take(1)

[(2, 1.0)]